# **Image Classification with CNNs**
*   **Objective:** Build and train a CNN to classify images from a dataset such as CIFAR-10 or ImageNet, etc.
*   **Key Concepts:** CNN architecture, data augmentation, transfer learning


In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau

In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0

In [4]:
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [5]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(train_images)

In [6]:
# Load pre-trained VGG16 model for transfer learning
base_model = tf.keras.applications.VGG16(input_shape=(32, 32, 3),
                                         include_top=False,
                                         weights='imagenet')

# Summary of the pre-trained model
base_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

# **Model-1**

In [11]:
# Fine-tune the last few layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Define the model architecture
model1 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                      

In [13]:
# Compile the model
model1.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model with data augmentation
history = model1.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    steps_per_epoch=len(train_images) / 64,
                    epochs=30,
                    validation_data=(test_images, test_labels),
                    callbacks=[checkpoint, reduce_lr])

# Evaluate the model
test_loss, test_acc = model1.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_acc}")

Epoch 1/30
782/781 [==============================] - ETA: 0s - loss: 2.3032 - accuracy: 0.1015
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


781/781 [==============================] - 42s 49ms/step - loss: 2.3032 - accuracy: 0.1015 - val_loss: 2.3028 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/30
781/781 [============================>.] - ETA: 0s - loss: 2.1463 - accuracy: 0.1774
Epoch 2: val_accuracy improved from 0.10000 to 0.21900, saving model to best_model.h5
781/781 [==============================] - 39s 50ms/step - loss: 2.1457 - accuracy: 0.1776 - val_loss: 2.8224 - val_accuracy: 0.2190 - lr: 0.0010
Epoch 3/30
782/781 [==============================] - ETA: 0s - loss: 1.5205 - accuracy: 0.4240
Epoch 3: val_accuracy improved from 0.21900 to 0.50420, saving model to best_model.h5
781/781 [==============================] - 39s 49ms/step - loss: 1.5205 - accuracy: 0.4240 - val_loss: 1.3698 - val_accuracy: 0.5042 - lr: 0.0010
Epoch 4/30
781/781 [============================>.] - ETA: 0s - loss: 1.3077 - accuracy: 0.5201
Epoch 4: val_accuracy improved from 0.50420 to 0.58960, saving model to best_model.h5
781/781 [=======

# **Model-2**

In [20]:
# Define CNN architecture
model2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_6 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_7 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                      

In [23]:
# Compile the model
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model2.fit(train_images, train_labels, epochs=30, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model2.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_acc}")

Epoch 1/30
313/313 [==============================] - 15s 30ms/step - loss: 1.7410 - accuracy: 0.3984 - val_loss: 3.3037 - val_accuracy: 0.2060
Epoch 2/30
313/313 [==============================] - 7s 23ms/step - loss: 1.2510 - accuracy: 0.5529 - val_loss: 1.2707 - val_accuracy: 0.5583
Epoch 3/30
313/313 [==============================] - 7s 23ms/step - loss: 1.0389 - accuracy: 0.6348 - val_loss: 0.9648 - val_accuracy: 0.6534
Epoch 4/30
313/313 [==============================] - 7s 22ms/step - loss: 0.9024 - accuracy: 0.6856 - val_loss: 1.0912 - val_accuracy: 0.6513
Epoch 5/30
313/313 [==============================] - 7s 23ms/step - loss: 0.8109 - accuracy: 0.7166 - val_loss: 0.7658 - val_accuracy: 0.7420
Epoch 6/30
313/313 [==============================] - 7s 23ms/step - loss: 0.7428 - accuracy: 0.7418 - val_loss: 0.7501 - val_accuracy: 0.7421
Epoch 7/30
313/313 [==============================] - 7s 23ms/step - loss: 0.6939 - accuracy: 0.7581 - val_loss: 0.6363 - val_accuracy: 0.780

# **Model-3**

In [14]:
# Load MobileNetV2 as base model
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')


9406464/9406464 [==============================] - 0s 0us/step


In [15]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

# Add classification head
model3 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 1, 1, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_8 (Dense)             (None, 512)               655872    
                                                                 
 batch_normalization_3 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                      

In [16]:
# Define learning rate schedule
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0005
    else:
        return 0.0001

# Compile the model
model3.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [LearningRateScheduler(lr_schedule)]

# Train the model
history = model3.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    steps_per_epoch=len(train_images) / 64,
                    epochs=30,
                    validation_data=(test_images, test_labels),
                    callbacks=callbacks)

# Evaluate the model
test_loss, test_acc = model3.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_acc}")

Epoch 1/30
781/781 [==============================] - 82s 68ms/step - loss: 2.0544 - accuracy: 0.3009 - val_loss: 2.0307 - val_accuracy: 0.2995 - lr: 0.0010
Epoch 2/30
781/781 [==============================] - 52s 67ms/step - loss: 1.2791 - accuracy: 0.5662 - val_loss: 1.7144 - val_accuracy: 0.4550 - lr: 0.0010
Epoch 3/30
781/781 [==============================] - 52s 67ms/step - loss: 1.0646 - accuracy: 0.6473 - val_loss: 1.3693 - val_accuracy: 0.5857 - lr: 0.0010
Epoch 4/30
781/781 [==============================] - 54s 69ms/step - loss: 0.9812 - accuracy: 0.6788 - val_loss: 1.8129 - val_accuracy: 0.5761 - lr: 0.0010
Epoch 5/30
781/781 [==============================] - 52s 67ms/step - loss: 0.9079 - accuracy: 0.7025 - val_loss: 1.0751 - val_accuracy: 0.7017 - lr: 0.0010
Epoch 6/30
781/781 [==============================] - 53s 68ms/step - loss: 0.8713 - accuracy: 0.7188 - val_loss: 1.1367 - val_accuracy: 0.6925 - lr: 0.0010
Epoch 7/30
781/781 [==============================] - 51s 

In [24]:
# Load CIFAR-10 dataset
(_, _), (test_images, test_labels) = cifar10.load_data()
test_images = test_images.astype('float32') / 255.0
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

# Define models with their names
models = [(model1, 'model1'), (model2, 'model2'), (model3, 'model3')]

# Evaluate model function
def evaluate_model(model, test_images, test_labels):
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
    return test_acc

# Evaluate models
for model, name in models:
    accuracy = evaluate_model(model, test_images, test_labels)
    print(f"{name}: Test Accuracy - {accuracy * 100:.2f}%")

model1: Test Accuracy - 76.49%
model2: Test Accuracy - 84.12%
model3: Test Accuracy - 85.03%
